# Building a Data Pipeline 

Collection of Code for the `Script Body` parameters of various `ExecuteScript` NiFi processors

## Query SeeClickFix API

In [ ]:
# always include these libraries when executing jython in NiFi
import java.io
from org.apache.commons.io import IOUtils
from java.nio.charset import StandardCharsets
from org.apache.nifi.processor.io import StreamCallback
from org.python.core.util import StringUtil


import urllib
import urllib2
import json

class ModJSON(StreamCallback):
    """
    Class to handle SCF API and NiFi operations
    """
    def __init__(self):
        pass
    def process(self, inputStream, outputStream):
        """
        Make request to SeeClickFix endpoint and write the response to a JSON file 

        Args: 
            - inputStream (): 
            - outputStream ():

        Returns:
            - FlowFile as a JSON filled with data from the API
        """
        try:
           # set location and page params 
           params = {
              "place_url": "downtown_district-of-columbia", 
              "per_page": "100"
           }
           # construct request URL
           url = f"https://seeclickfix.com/api/v2/issues?{urllib.urlencode(params)}"
           # get data at URL
           res = urllib2.urlopen(url).read()
           # convert to useable format
           data = json.loads(res)
           # write flowfile as JSON with data from API
           outputStream.write(bytearray(json.dumps(data, indent=4).encode("utf-8")))
        except:
           # in case of failure, set error state to ensure the Failure relationship is triggered
           global errorOccurred
           errorOccurred = True
           outputStream.write(bytearray(json.dumps(data, indent=4).encode("utf-8")))

errorOccurred=False

# try and retrieve a FlowFile
flowFile = session.get()

if (flowFile != None):
  # if found, get data from API
  flowFile = session.write(flowFile, ModJSON())
  #flowFile = session.putAttribute(flowFile) ## SM: commented out for now, dont know why
  if(errorOccurred):
    # send to failure relationship
    session.transfer(flowFile, REL_FAILURE)
else:
    # send to success relationship
    session.transfer(flowFile, REL_SUCCESS)